### Parameter configuration for SDO
#### Used for Table 1

Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from clawpack import pyclaw
import sys
from piston_Lagrangian import setup
from matplotlib import animation
from IPython.display import HTML
plt.rcParams['text.usetex'] = True
plt.rcParams['animation.ffmpeg_path'] = r'/usr/local/bin/ffmpeg'
from utils import max_error

Compute a reference solution

In [ ]:
p = 20 #Number of  periods
p_domain = 30 #Number of periods in the domain
xmax1 = p_domain*2*np.pi
N = 250 #Cells per period at final time
mx1 = p_domain*N#int(N*(xmax1/p)) #Number of grid points
M = 0.4 
piston_freq=1.
tfinal = p*2*np.pi
piston_problem1 = setup(tfinal=tfinal, xmax=xmax1, mx=mx1, M=M, CFL=0.8,limiting=1,solver_type='sharpclaw',
order=2, time_integrator='Heun', euler_RS='euler', nout=100,
piston_freq=piston_freq)
piston_problem1.verbosity = 0
piston_problem1.run()
piston_problem1.solver.__del__() #Deallocation of Fortran arrays

In [ ]:
#Combinations of s and d
list_functions = ["Engsig-Karup2","Mayer"]
list_combinations = [(0,0),(0,1),(1,0),(1,1)]
combination_names = ["sA,dA","sA,dB","sB,dA","sB,dB"]

Computing error

In [ ]:
def comparison_slowing_before_damping(stop_slowing=25*np.pi ,stop_absorbing=25*np.pi,
                                      list_functions=list_functions, list_combinations=list_combinations,
                                      combination_names=combination_names):
    p = 20 #Number of  periods
    p_domain = 30 #Number of periods in the domain
    xmax_expected = p*2*np.pi #Maximum value of x for nontrivial solution
    xmax2 = p_domain*2*np.pi
    N = 250#200 #Cells per period at final time
    mx2 = p_domain*N#int(N*(xmax1/p)) #Number of grid points
    M = 0.4 
    tfinal = p*2*np.pi#p*2*np.pi
    start_slowing = 20*np.pi
    stop_slowing = stop_slowing
    start_absorbing = 20*np.pi
    stop_absorbing = stop_absorbing
    local_list_errors = []
    for i in range(len(list_combinations)):
        slowing_fun, damping_fun = list_functions[list_combinations[i][0]], list_functions[list_combinations[i][1]]
        piston_problem2 = setup(tfinal=tfinal, xmax=xmax2, mx=mx2, M=M, CFL=0.8,limiting=1, #1 TVD, 2 WENO
                                solver_type='sharpclaw', order=2, time_integrator='Heun',euler_RS='euler_slowing',
                                start_slowing = start_slowing, stop_slowing = stop_slowing,
                                start_absorbing=start_absorbing, stop_absorbing=stop_absorbing,
                                far_field_damping_rate=30., far_field_damping=True, nout=100,
                                sigma_damping_type=damping_fun,
                                sigma_slowing_type=slowing_fun, 
                                scalar_far_field=False,
                                #implicit_integrator=False,
                                #integral_source=False,
                                relaxation_method=False,
                                matrix_filter=False
                                )
        piston_problem2.verbosity = 0
        piston_problem2.run()
        piston_problem2.solver.__del__() #Deallocation of Fortran arrays
        indx = np.where(piston_problem2.grid.x.centers<=start_slowing)
        local_list_errors.append(max_error(piston_problem1,piston_problem2, indx=indx))
    return local_list_errors


In [ ]:
list_errors = []
for i in [0.25,0.5,1.,2.]:
    list_errors.append(comparison_slowing_before_damping(stop_slowing=(20+i)*np.pi, stop_absorbing=(20+i)*np.pi))

In [ ]:
list_errors

In [ ]:
#Print list errors using tabulate rounding to 3 decimals and using scientific notation
from tabulate import tabulate
print(tabulate(list_errors, tablefmt="latex", floatfmt=".2e", headers=combination_names))
